# Программирование для всех (основы работы с Python)

*Алла Тамбовцева, НИУ ВШЭ*

## Работа с датафреймами `pandas`: часть 2

* Переименование и удаление столбцов
* Выбор нескольких столбцов и строк, методы `.loc` и `.iloc`
* Фильтрация строк
* Добавление новых столбцов
* Группировка и агрегирование

### Загрузка данных

Импортируем библиотеку `pandas` и загрузим данные из файла `Salaries.csv`, с которым мы работали в прошлый раз.

In [1]:
import pandas as pd

In [2]:
dat = pd.read_csv("Salaries.csv")
dat.head()

,Unnamed: 0,rank,discipline,yrs.since.phd,yrs.service,sex,salary
0,1,Prof,B,19,18,Male,139750
1,2,Prof,B,20,16,Male,173200
2,3,AsstProf,B,4,3,Male,79750
3,4,Prof,B,45,39,Male,115000
4,5,Prof,B,40,41,Male,141500


Напоминание о переменных:

* `rank`: должность;
* `discipline`: тип преподаваемой дисциплины (A – теоретическая, B – практическая);
* `yrs.since.phd`: число лет с момента получения степени PhD;
* `yrs.service`: число лет опыта работы;
* `sex`: пол;
* `salary`: заработная плата за 9 месяцев, в долларах.

### Переименование и удаление столбцов

Избавимся от длинных названий с точкой – переименуем столбцы `yrs.service` и `yrs.since.phd`. Воспользуемся методом `.rename()` и поместим в него словарь, где ключами являются старые названия столбцов, а значениями – новые:

In [3]:
dat.rename({"yrs.since.phd" : "phd", "yrs.service" : "service"})
dat.head()

,Unnamed: 0,rank,discipline,yrs.since.phd,yrs.service,sex,salary
0,1,Prof,B,19,18,Male,139750
1,2,Prof,B,20,16,Male,173200
2,3,AsstProf,B,4,3,Male,79750
3,4,Prof,B,45,39,Male,115000
4,5,Prof,B,40,41,Male,141500


Как можно заметить, с `dat` никаких изменений не произошло. Почему? Метод `.rename()` по умолчанию работает со строками, а не со столбцами, поэтому здесь нужно явно указать название аргумента `columns`:

In [4]:
dat.rename(columns = {"yrs.since.phd" : "phd", "yrs.service" : "service"})
dat.head()

,Unnamed: 0,rank,discipline,yrs.since.phd,yrs.service,sex,salary
0,1,Prof,B,19,18,Male,139750
1,2,Prof,B,20,16,Male,173200
2,3,AsstProf,B,4,3,Male,79750
3,4,Prof,B,45,39,Male,115000
4,5,Prof,B,40,41,Male,141500


Забавно, но изменений всё ещё нет. Это связано с тем, что, несмотря на то, что тип `DataFrame` является изменяемым, многие методы на датафреймах по умолчанию работают «в безопасном режиме», то есть возвращают изменённую копию датафрейма, не изменяя оригинал. Чтобы сохранить изменения без переприсваивания через `=`, добавим аргумент `inplace = True`:

In [5]:
# аналог менее изящного 
# dat = dat.rename(columns = {"yrs.since.phd" : "phd", "yrs.service" : "service"})

dat.rename(columns = {"yrs.since.phd" : "phd", "yrs.service" : "service"}, inplace = True)
dat.head()

,Unnamed: 0,rank,discipline,phd,service,sex,salary
0,1,Prof,B,19,18,Male,139750
1,2,Prof,B,20,16,Male,173200
2,3,AsstProf,B,4,3,Male,79750
3,4,Prof,B,45,39,Male,115000
4,5,Prof,B,40,41,Male,141500


Изменения в `dat` сохранились. Теперь удалим лишний столбец `Unnamed: 0` с номером строки. Метод `.drop()` работает по схожей схеме, по умолчанию удаляет строки, для работы со столбцами указываем в аргументе `columns` список названий столбцов для удаления:

In [6]:
# и снова inplace = True

dat.drop(columns = ["Unnamed: 0"], inplace = True)
dat.head()

,rank,discipline,phd,service,sex,salary
0,Prof,B,19,18,Male,139750
1,Prof,B,20,16,Male,173200
2,AsstProf,B,4,3,Male,79750
3,Prof,B,45,39,Male,115000
4,Prof,B,40,41,Male,141500


### Выбор нескольких столбцов и строк, методы `.loc` и `.iloc`

Если нам нужно сразу несколько столбцов (маленький датафрейм на основе старого), то названия столбцов необходимо оформить в виде списка и указать его в квадратных скобках:

In [7]:
small = dat[["sex", "rank", "salary"]] 
small

,sex,rank,salary
0,Male,Prof,139750
1,Male,Prof,173200
2,Male,AsstProf,79750
3,Male,Prof,115000
4,Male,Prof,141500
...,...,...,...
392,Male,Prof,103106
393,Male,Prof,150564
394,Male,Prof,101738
395,Male,Prof,95329


Кроме того, в `pandas` на датафреймах определены два метода:
    
* метод `loc` для выбора строк/столбцов по названиям (*location*).    
* метод `iloc` для выбора строк/столбцов по индексам (*index location*);

Если нам нужно несколько столбцов подряд, начиная с одного названия и заканчивая другим, можно воспользоваться методом `.loc`:

In [8]:
dat.loc[:, "rank" : "service"]

,rank,discipline,phd,service
0,Prof,B,19,18
1,Prof,B,20,16
2,AsstProf,B,4,3
3,Prof,B,45,39
4,Prof,B,40,41
...,...,...,...,...
392,Prof,A,33,30
393,Prof,A,31,19
394,Prof,A,42,25
395,Prof,A,25,15


Метод `.loc` используется для выбора определенных строк и столбцов, поэтому в квадратных скобках образуется запись через запятую: на первом месте условия для строк, на втором – для столбцов. Здесь нас интересуют все строки (полный срез через `:`) и конкретные столбцы, с `rank` по `service` включительно.

Если бы мы хотели выбрать строки с 0 по 10 и столбцы с `rank` по `service`, тоже бы пригодился метод `.loc`:

In [9]:
dat.loc[0:10, "rank" : "service"]

,rank,discipline,phd,service
0,Prof,B,19,18
1,Prof,B,20,16
2,AsstProf,B,4,3
3,Prof,B,45,39
4,Prof,B,40,41
5,AssocProf,B,6,6
6,Prof,B,30,23
7,Prof,B,45,45
8,Prof,B,21,20
9,Prof,B,18,18


**Внимание:** хотя в `.loc` мы вроде как задействуем обычные питоновские срезы, внутри этого метода срезы включают как левый, так и правый конец. Так, в примере выше были выбраны строки по 10-ую включительно и столбец `service` также был включен.

Иногда может возникнуть необходимость выбрать столбец по его порядковому номеру. Например, когда названий столбцов нет как таковых или когда названия слишком длинные, а переименовывать их нежелательно. Сделать это можно с помощью метода `.iloc` (*i* – от *index*). Выберем строки с 0 по 9 и столбцы с 0 по 3:

In [10]:
dat.iloc[0:10, 0:4]

,rank,discipline,phd,service
0,Prof,B,19,18
1,Prof,B,20,16
2,AsstProf,B,4,3
3,Prof,B,45,39
4,Prof,B,40,41
5,AssocProf,B,6,6
6,Prof,B,30,23
7,Prof,B,45,45
8,Prof,B,21,20
9,Prof,B,18,18


**Внимание:** в методе `.iloc`, поскольку работа идет с обычными числовыми индексами (как в списках и кортежах), правый конец среза исключается. Поэтому в примере выше 10-я строка и 4-ый столбец показаны не были.

Если в `.iloc` вписать только одно число, по умолчанию будет выдана строка с таким номером:

In [11]:
dat.iloc[2]

rank          AsstProf
discipline           B
phd                  4
service              3
sex               Male
salary           79750
Name: 2, dtype: object

Это будет объект типа `pandas Series`, своего рода срез датафрейма:

In [12]:
type(dat.iloc[2])

pandas.core.series.Series

**Дополнительно.** Обратите внимание: из-за наличия строковых значений в строке, мы получили объект `pandas Series` с типом данных `object`, строковый тип всегда сильнее числового и вытесняет его. Что интересно, если из этой последовательности `Series` извлечь отдельные значения, они будут правильного типа (как в исходном датафрейме):

In [13]:
# выбор столбца salary из выбранной строки
# тип integer

dat.iloc[1, :]["salary"]

173200

### Фильтрация строк

Часто при работе с датафреймом нас не интересует выбор отдельных строк по названию или номеру, а интересует фильтрация наблюдений – выбор строк датафрейма, которые удовлетворяют определенному условию. Для этого интересующее нас условие необходимо указать в квадратных скобках. Например, выберем только те строки, которые соответствуют сотрудникам с опытом работы более 10 лет:

In [14]:
dat[dat["service"] > 10]

,rank,discipline,phd,service,sex,salary
0,Prof,B,19,18,Male,139750
1,Prof,B,20,16,Male,173200
3,Prof,B,45,39,Male,115000
4,Prof,B,40,41,Male,141500
6,Prof,B,30,23,Male,175000
...,...,...,...,...,...,...
391,Prof,A,30,19,Male,151292
392,Prof,A,33,30,Male,103106
393,Prof,A,31,19,Male,150564
394,Prof,A,42,25,Male,101738


Почему нельзя было написать проще, то есть `dat["service"] > 10`? Давайте напишем, и посмотрим, что получится:

In [15]:
dat["service"] > 10

0       True
1       True
2      False
3       True
4       True
       ...  
392     True
393     True
394     True
395     True
396    False
Name: service, Length: 397, dtype: bool

Что мы увидели? Просто результат проверки условия для каждой строки датафрейма, набор из `True` и `False`. Когда мы подставляем это выражение в квадратные скобки, Python выбирает из `dat` те строки, где выражение принимает значение `True`.

Все символьные операторы для объединения условий работают как обычно, только тут важно не забывать про круглые скобки вокруг каждого условия. Например, выберем сотрудников женского пола со стажем более 10 лет:

In [16]:
# одновременное выполнение условий
dat[(dat["service"] > 10) & (dat["sex"] == "Female")]

,rank,discipline,phd,service,sex,salary
9,Prof,B,18,18,Female,129000
19,Prof,A,39,36,Female,137000
47,Prof,B,23,19,Female,151768
48,Prof,B,25,25,Female,140096
63,AssocProf,B,11,11,Female,103613
68,Prof,B,17,17,Female,111512
84,Prof,B,17,18,Female,122960
103,Prof,B,20,14,Female,127512
123,AssocProf,A,25,22,Female,62884
148,Prof,B,36,26,Female,144651


А теперь выберем профессоров или доцентов:

In [17]:
# или одно верно, или другое, или оба
dat[(dat["rank"] == "Prof") | (dat["rank"] == "AssocProf")]

,rank,discipline,phd,service,sex,salary
0,Prof,B,19,18,Male,139750
1,Prof,B,20,16,Male,173200
3,Prof,B,45,39,Male,115000
4,Prof,B,40,41,Male,141500
5,AssocProf,B,6,6,Male,97000
...,...,...,...,...,...,...
391,Prof,A,30,19,Male,151292
392,Prof,A,33,30,Male,103106
393,Prof,A,31,19,Male,150564
394,Prof,A,42,25,Male,101738


Однако иногда для формулировки условий стандартных операторов (`&`, `|`, `^`) недостаточно. Чтобы не писать длинную последовательность с `|`, можно проверить, входят ли значения в некотором столбце в список, с помощью метода `.isin()`:

In [18]:
# .isin() возвращает True/False
dat[dat["rank"].isin(["Prof", "AssocProf"])]

,rank,discipline,phd,service,sex,salary
0,Prof,B,19,18,Male,139750
1,Prof,B,20,16,Male,173200
3,Prof,B,45,39,Male,115000
4,Prof,B,40,41,Male,141500
5,AssocProf,B,6,6,Male,97000
...,...,...,...,...,...,...
391,Prof,A,30,19,Male,151292
392,Prof,A,33,30,Male,103106
393,Prof,A,31,19,Male,150564
394,Prof,A,42,25,Male,101738


Для числовых данных это тоже работает:

In [19]:
dat[dat["service"].isin([10, 20])]

,rank,discipline,phd,service,sex,salary
8,Prof,B,21,20,Male,119250
16,Prof,B,19,20,Male,101000
82,Prof,B,22,20,Male,144640
94,Prof,B,21,20,Male,123683
104,AssocProf,A,18,10,Male,83850
141,AssocProf,A,15,10,Male,81500
153,AssocProf,B,12,10,Female,103994
173,Prof,B,20,20,Male,134185
186,AssocProf,B,13,10,Female,103750
187,Prof,B,18,10,Male,107500


**Важно.** Метод `.isin()` в случае числовых значений не создаёт интервалы из целых чисел, он проверяет равенство каждому числу в отдельности.

Вернемся к столбцу из текстовых значений. Если бы разных значений с подстрокой `"Prof"` в `rank` было много, было бы неудобно прописывать через `|` однотипные условия для каждой должности. Тогда логично было бы воспользоваться методом, который позволяет выбрать все строки, где в ячейке с текстом встречается слово `"Prof"`. Такой метод есть – это метод на строках `.contains()`, который возвращает `True`, если некоторая подстрока входит в строку, и `False` – в противном случае.

In [20]:
# в нашем случае это все строки, так как везде есть Prof
dat[dat["rank"].str.contains("Prof")]

,rank,discipline,phd,service,sex,salary
0,Prof,B,19,18,Male,139750
1,Prof,B,20,16,Male,173200
2,AsstProf,B,4,3,Male,79750
3,Prof,B,45,39,Male,115000
4,Prof,B,40,41,Male,141500
...,...,...,...,...,...,...
392,Prof,A,33,30,Male,103106
393,Prof,A,31,19,Male,150564
394,Prof,A,42,25,Male,101738
395,Prof,A,25,15,Male,95329


Можно попросить не учитывать регистр:

In [21]:
# case = False
# prof с буквами любого регистра

dat[dat["rank"].str.contains("prof", case = False)]

,rank,discipline,phd,service,sex,salary
0,Prof,B,19,18,Male,139750
1,Prof,B,20,16,Male,173200
2,AsstProf,B,4,3,Male,79750
3,Prof,B,45,39,Male,115000
4,Prof,B,40,41,Male,141500
...,...,...,...,...,...,...
392,Prof,A,33,30,Male,103106
393,Prof,A,31,19,Male,150564
394,Prof,A,42,25,Male,101738
395,Prof,A,25,15,Male,95329


**Дополнительно.** А если наоборот, нам нужно отрицание – все строки, которые относятся к чему угодно, только не к `"Prof"`? Можно воспользоваться оператором `~` для отрицания и поставить его перед всем условием в скобках:

In [22]:
# таких нет, у всех Prof встречается в названии должности
dat[~dat["rank"].str.contains("Prof")]

,rank,discipline,phd,service,sex,salary


В целом, если вы владеете регулярными выражениями, для поиска по частичным совпадениям достаточно метода `.contains()`, но, как частные случаи, в этом наборе методов есть функции `.startswith()` и `.endswith()`, которые проверяют, начинается ли/заканчивается ли строка на определённую последовательность символов.

### Добавление новых столбцов

Если нам нужно добавить столбец из одинаковых значений (например, указание на версию датафрейма), создавать список определённой длины необязательно, Pandas умеет «растягивать» значение на все ячейки:

In [23]:
# одинаковые значения

dat["version"] = "v.01"
dat.head()

,rank,discipline,phd,service,sex,salary,version
0,Prof,B,19,18,Male,139750,v.01
1,Prof,B,20,16,Male,173200,v.01
2,AsstProf,B,4,3,Male,79750,v.01
3,Prof,B,45,39,Male,115000,v.01
4,Prof,B,40,41,Male,141500,v.01


Если нам нужно создать новый столбец на основе старого с помощью базовых операций (умножение, деление, возведение в степень, сложение двух столбцов), никаких специальных функций не требуется, можем пользоваться тем, что операции на типе `pandas Series` векторизованы, как на массивах:

In [24]:
# делим столбец на 1000

dat["salary_th"] = dat["salary"] / 1000
dat.head()

,rank,discipline,phd,service,sex,salary,version,salary_th
0,Prof,B,19,18,Male,139750,v.01,139.75
1,Prof,B,20,16,Male,173200,v.01,173.20
2,AsstProf,B,4,3,Male,79750,v.01,79.75
3,Prof,B,45,39,Male,115000,v.01,115.00
4,Prof,B,40,41,Male,141500,v.01,141.50


Если нам нужно добавить бинарный столбец на основе простого условия, можем проверить это условие с помощью базовых операторов:

In [25]:
dat["sex"] == "Female"

0      False
1      False
2      False
3      False
4      False
       ...  
392    False
393    False
394    False
395    False
396    False
Name: sex, Length: 397, dtype: bool

И преобразовать тип полученного столбца в целочисленный:

In [26]:
# бинарный столбец с полом сотрудника

dat["female"] = (dat["sex"] == "Female").astype(int)
dat.head()

,rank,discipline,phd,service,sex,salary,version,salary_th,female
0,Prof,B,19,18,Male,139750,v.01,139.75,0
1,Prof,B,20,16,Male,173200,v.01,173.20,0
2,AsstProf,B,4,3,Male,79750,v.01,79.75,0
3,Prof,B,45,39,Male,115000,v.01,115.00,0
4,Prof,B,40,41,Male,141500,v.01,141.50,0


Можем проверить, что в столбце есть как 0, так и 1. Метод `.value_counts()` построит таблицу частот:

In [27]:
# точнее, это pandas Series с частотами

dat["female"].value_counts()

0    358
1     39
Name: female, dtype: int64

Если нам нужен столбец с двумя значениями, но эти значения не 0 и 1, можем написать свою простую lambda-функцию для перекодировки и применить её к уже имеющемуся столбцу через метод `.apply()`:

In [28]:
# столбец с двумя значениями
# заменяем A на Theory, B на Practice

dat["course"] = dat["discipline"].apply(lambda x: "Theory" if x == "A" else "Practice")
dat.head()

,rank,discipline,phd,service,sex,salary,version,salary_th,female,course
0,Prof,B,19,18,Male,139750,v.01,139.75,0,Practice
1,Prof,B,20,16,Male,173200,v.01,173.20,0,Practice
2,AsstProf,B,4,3,Male,79750,v.01,79.75,0,Practice
3,Prof,B,45,39,Male,115000,v.01,115.00,0,Practice
4,Prof,B,40,41,Male,141500,v.01,141.50,0,Practice


Метод `.apply()` – аналог базового `map()`, только ещё более эффективный при работе с датафреймами. Он применяет написанную функцию к каждой ячейке в столбце (хотя его можно написать и для строк тоже, есть аргумент `axis`).

Конечно, если функция более объёмная, её лучше задать отдельно, а в `.apply()` подставить её название. Перекодируем значения должностей:

* `Prof` в 1;
* `AssocProf` в 2;
* `AsstProf` в 3.

In [29]:
# пишем функцию и применяем ее
def get_num_rank(x):
    if x == "Prof":
        y = 1
    elif x == "AssocProf":
        y = 2
    elif x == "AsstProf":
        y = 3
    else:
        y = None
    return y

dat["rank_num"] = dat["rank"].apply(get_num_rank)
dat.head()

,rank,discipline,phd,service,sex,salary,version,salary_th,female,course,rank_num
0,Prof,B,19,18,Male,139750,v.01,139.75,0,Practice,1
1,Prof,B,20,16,Male,173200,v.01,173.20,0,Practice,1
2,AsstProf,B,4,3,Male,79750,v.01,79.75,0,Practice,3
3,Prof,B,45,39,Male,115000,v.01,115.00,0,Practice,1
4,Prof,B,40,41,Male,141500,v.01,141.50,0,Practice,1


**Дополнительно 1.** Представленный выше способ – универсальный, он подойдёт для действительно больших функций (для обработки текста, например), а в нашем случае столбец с перекодированными должностями можно было создать ещё проще, вообще без функций. В Pandas есть метод  `.map()`, которая умеет выполнять замену значений, заданную с помощью словаря (ключи – старые значения, значения – новые, похоже на `.rename()` по своей логике):

In [30]:
# создаем словарь соответствий и задействуем

D = {"Prof" : 1, "AssocProf" : 2, "AsstProf" : 3}
dat["rank_new"] = dat["rank"].map(D)

dat.head()

,rank,discipline,phd,service,sex,salary,version,salary_th,female,course,rank_num,rank_new
0,Prof,B,19,18,Male,139750,v.01,139.75,0,Practice,1,1
1,Prof,B,20,16,Male,173200,v.01,173.20,0,Practice,1,1
2,AsstProf,B,4,3,Male,79750,v.01,79.75,0,Practice,3,3
3,Prof,B,45,39,Male,115000,v.01,115.00,0,Practice,1,1
4,Prof,B,40,41,Male,141500,v.01,141.50,0,Practice,1,1


**Дополнительно 2.** В задачах, связанных с прикладным анализом данных, часто возникает необходимость представить качественную (текстовую) информацию в числовой форме. Для этого часто прибегают к процедуре создания фиктивных переменных (дамми-переменных), которая также называется **one-hot encoding**. В ходе этой процедуры для каждого уникального `x` в столбце создаётся новый бинарный столбец, где 1 ставится, если значение в строке равно `x`, и 0 – иначе. Для примера создадим набор дамми-переменных для должности:

In [31]:
# первый и второй сотрудник – профессор,
# третий – преподаватель

new = pd.get_dummies(dat["rank"])
new.head()

,AssocProf,AsstProf,Prof
0,0,0,1
1,0,0,1
2,0,1,0
3,0,0,1
4,0,0,1


Единственное, стоит учесть, что функция `get_dummies()` создаёт новый датафрейм с бинарными столбцами, а не добавляет их в старый. Поэтому исходный датафрейм нужно склеить с новым. Воспользуемся функцией `concat()` и подадим этой функции на вход список датафреймов, которые необходимо склеить:

In [32]:
dat = pd.concat([dat, new], axis = 1)
dat.head()

,rank,discipline,phd,service,sex,salary,version,salary_th,female,course,rank_num,rank_new,AssocProf,AsstProf,Prof
0,Prof,B,19,18,Male,139750,v.01,139.75,0,Practice,1,1,0,0,1
1,Prof,B,20,16,Male,173200,v.01,173.20,0,Practice,1,1,0,0,1
2,AsstProf,B,4,3,Male,79750,v.01,79.75,0,Practice,3,3,0,1,0
3,Prof,B,45,39,Male,115000,v.01,115.00,0,Practice,1,1,0,0,1
4,Prof,B,40,41,Male,141500,v.01,141.50,0,Practice,1,1,0,0,1


По умолчанию функция `concat()` склеивает датафреймы по строкам (доклеивает новый датафрейм снизу), значение аргумента `axis` равно 0. Нам же нужно было склеить датафреймы по столбцам (доклить `new` справа от `df`), поэтому мы изменили ось `axis` на 1. Функция `concat()` объединяет датафреймы «как есть», просто присоединяя одну таблицу к другой снизу или справа, для более продвинутого объединения нужен метод `.merge()`. Он реализует разные варианты объединений по какому-то столбцу, например, с id, по аналогии с операциями над базами данных (`INNER JOIN`, `OUTER JOIN` и подобные).

### Группировка и агрегирование

Рассмотрим группировку строк в датафрейме по фиксированному признаку. Для группировки используется метод `.groupby()`, в качестве аргумента указывается столбец, по которому группируем:

In [33]:
dat.groupby("sex")

Этот метод возвращает скрытый объект специального типа, он временно хранится в какой-то ячейке памяти. Внутри этот объект похож на список кортежей, где на первом месте указаны значения показателя группировки, а на втором – датафрейм из строк, соответствующих этому значению.

In [ ]:
#print(*dat.groupby("sex"))

Так, если раскомментировать код выше, мы получим пары:

*  `female` и датафрейм с отфильтрованными строками, соответствующими сотрудникам женского пола;
* `male` и датафрейм с отфильтрованными строками, соответствующими сотрудникам мужского пола.

Если групп много (а не две, как здесь), этот результат удобно использовать для выгрузки данных по группам. Сделаем множественный перебор в `for` и сохраним датафрейм по каждой группе в CSV-файл:

In [34]:
for name, data in dat.groupby("sex"):
    data.to_csv(name + ".csv")

Что важно, объект, возвращаемый `.groupby()`, не просто список пар. Это особый тип данных в Pandas, из которого можно извлекать столбцы по названиям, как в датафреймах. Так, если напишем следующий код

In [37]:
dat.groupby("sex")["salary"].mean()

sex
Female    101002.410256
Male      115090.418994
Name: salary, dtype: float64

Pandas выберет из каждого датафрейма, соответствующего группе, столбец `salary` и вычислит по нему среднее. Другими словами, можно сочетать группировку и агрегирование. 

Что удобно, за один раз можно получить сразу несколько описательных статистик сразу:

In [38]:
dat.groupby("sex")["salary"].describe()

,count,mean,std,min,25%,50%,75%,max
sex,,,,,,,,
Female,39.0,101002.410256,25952.127317,62884.0,77250.0,103750.0,117002.50,161101.0
Male,358.0,115090.418994,30436.927344,57800.0,92000.0,108043.0,134863.75,231545.0


Выберем два столбца, сравним характеристики стажа и зарплаты мужчин и женщин:

In [39]:
dat.groupby("sex")[["service", "salary"]].describe()

service                                                   salary  \
         count       mean        std  min  25%   50%   75%   max  count   
sex                                                                       
Female    39.0  11.564103   8.813252  0.0  4.0  10.0  17.5  36.0   39.0   
Male     358.0  18.273743  13.226234  0.0  7.0  18.0  27.0  60.0  358.0   

                                                                            \
                 mean           std      min      25%       50%        75%   
sex                                                                          
Female  101002.410256  25952.127317  62884.0  77250.0  103750.0  117002.50   
Male    115090.418994  30436.927344  57800.0  92000.0  108043.0  134863.75   

                  
             max  
sex               
Female  161101.0  
Male    231545.0

Можем выполнить группировку по двум показателям сразу – сгруппируем строки по полу и типу дисциплины:

In [40]:
# внутри groupby() список
dat.groupby(["sex", "discipline"])[["service", "salary"]].mean()

service         salary
sex    discipline                          
Female A           11.444444   89064.944444
       B           11.666667  111234.523810
Male   A           20.889571  110699.981595
       B           16.087179  118760.374359

**Дополнительно 1.** Если нас интересует набор определенных функций, а не готовый выбор тех, что в `.describe()`, пригодится метод `.agg()`, сокращение от *aggregate*. Внутри `.agg()` можно указать перечень функций в виде списка:

In [41]:
dat.groupby("sex")[["service", "salary"]].agg(["mean", "median"])

service                salary          
             mean median           mean    median
sex                                              
Female  11.564103   10.0  101002.410256  103750.0
Male    18.273743   18.0  115090.418994  108043.0

Если применяемые функции уже существуют в Pandas, их названия указываются в кавычках.

Метод `.agg()` довольно гибкий. Иногда возникает необходимость для одних данных использовать одни описательные статистики, а для других – другие. Так, если медиана и среднее сильно различаются между собой, это свидетельствует о наличии нехарактерных значений (если среднее сильно больше медианы, нетипично больших, если среднее сильно меньше медианы, нетипично маленьких). А при наличии нехарактерных значений среднее арифметическое уже не является показательным, это неустойчивая оценка среднего. 

В нашем случае сильных различий не наблюдается, но давайте для заработной платы посчитаем медиану, а для стажа – среднее. Запишем это в виде словаря (ключи – столбцы, значения – функции, которые нужно к этим столбцам применить):

In [42]:
dat.groupby("discipline").agg({"salary" : "median", 
                              "service" : "mean"})

,salary,service
discipline,,
A,104350.0,19.950276
B,113018.5,15.657407


**Дополнительно 2.** И, напоследок, ещё одно применение метода `.agg()`. Внутри этого метода можно указать свою функцию для агрегирования. Например, в Pandas нет готовой функции для вычисления размаха (максимальное значение минус минимальное), поэтому её можно написать самостоятельно:

In [43]:
dat.groupby("discipline")["salary"].agg(lambda x: max(x) - min(x))

discipline
A    147700
B    163986
Name: salary, dtype: int64

Если бы функция была более сложной, её можно было бы написать через `def`, а в `.agg()` просто указать её название, причём уже без кавычек:

In [44]:
def my_range(x):
    return max(x) - min(x)

dat.groupby("discipline")["salary"].agg(my_range)

discipline
A    147700
B    163986
Name: salary, dtype: int64